# Llama 3.2 fine tuning with color

2024-12-10 11:56

Fine tuning Llama instruct 3.2 with the color data set using unsloth. Good results but it might just be memorization because we didn't split the data set. Needs to run with test set and mid-training evaluation, but it looks promising.

In [1]:
!apt-get install build-essential -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  binutils binutils-common binutils-x86-64-linux-gnu bzip2 cpp cpp-11 dirmngr
  dpkg-dev fakeroot g++ g++-11 gcc gcc-11 gcc-11-base gnupg gnupg-l10n
  gnupg-utils gpg-agent gpg-wks-client gpg-wks-server gpgsm
  libalgorithm-diff-perl libalgorithm-diff-xs-perl libalgorithm-merge-perl
  libasan6 libatomic1 libbinutils libcc1-0 libctf-nobfd0 libctf0 libdpkg-perl
  libfakeroot libfile-fcntllock-perl libgcc-11-dev libgomp1 libisl23 libitm1
  libksba8 liblocale-gettext-perl liblsan0 libmpc3 libmpfr6 libnpth0
  libquadmath0 libstdc++-11-dev libtsan0 libubsan1 lto-disabled-list make
  patch pinentry-curses xz-utils
Suggested packages:
  binutils-doc bzip2-doc cpp-doc gcc-11-locales dbus-user-session
  pinentry-gnome3 tor debian-keyring g++-multilib g++-11-multilib gcc-11-doc
  gcc-multilib manpages-dev autoconf automake libtool flex bison gdb gcc

In [11]:
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found existing installation: unsloth 2024.12.4
Uninstalling unsloth-2024.12.4:
  Successfully uninstalled unsloth-2024.12.4
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-k083jc8z/unsloth_a8b4682916094aeb93f240871aad12f8
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-k083jc8z/unsloth_a8b4682916094aeb93f240871aad12f8
  Resolved https://github.com/unslothai/unsloth.git to commit df808d074d307d396b8c04640ef51375d68f2ef0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2024.12.4-py3-none-any.whl size=173562 sha256=3ac630a71d5f4f9ed0787ed157a5ea98c5eb7d2c0f21116d20e22532c64f42d6
  Stored in directory: /tmp/pip-ephem-wheel-cache-7tva8tmd/wheels/d1/17/05/850ab10c33284a4763b0595cd8ea9d01fce6e221cac24b3c01
Successfully built unsloth


In [12]:
import os
import numpy as np
import pandas as pd

import torch
from trl import SFTTrainer
from transformers import TrainingArguments, TextStreamer
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel
from datasets import Dataset
from unsloth import is_bfloat16_supported

# Saving model
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Warnings
import warnings
warnings.filterwarnings("ignore")

In [13]:
max_seq_length = 5020
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-1B-bnb-4bit",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
    use_rslora=True,
    use_gradient_checkpointing="unsloth",
    random_state = 32,
    loftq_config = None,
)
print(model.print_trainable_parameters())

==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: NVIDIA H100 NVL. Max memory: 93.003 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
trainable params: 11,272,192 || all params: 1,247,086,592 || trainable%: 0.9039
None


In [14]:
!apt install zip -y
!rm -rf data-rb-size
!mkdir -p data-rb-size
!wget "https://www.dropbox.com/scl/fi/q2qp4fyi37qkksgvyjco6/data-rb-size.zip?rlkey=37cpoqqpcykwgksqnqforij8u&dl=1" -O model.zip
!unzip model.zip -d data-rb-size

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
zip is already the newest version (3.0-12build2).
0 upgraded, 0 newly installed, 0 to remove and 28 not upgraded.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


--2024-12-10 10:54:06--  https://www.dropbox.com/scl/fi/q2qp4fyi37qkksgvyjco6/data-rb-size.zip?rlkey=37cpoqqpcykwgksqnqforij8u&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.13.18, 2620:100:6057:18::a27d:d12
Connecting to www.dropbox.com (www.dropbox.com)|162.125.13.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd2b9fafdeb908c5b582f7b7515.dl.dropboxusercontent.com/cd/0/inline/Cf_mzRzZL-LTRIx9XH18Qed1sdypm96q5lO1hdkM444wyn2S_CA0lwjqgW8nVnudzC2B9DlF-odFRbMk9YZ9OJTQLSlTyR66TM8o0EnEFouab4Dkn_ppqBASqb8NFmqoUgM/file?dl=1# [following]
--2024-12-10 10:54:08--  https://ucd2b9fafdeb908c5b582f7b7515.dl.dropboxusercontent.com/cd/0/inline/Cf_mzRzZL-LTRIx9XH18Qed1sdypm96q5lO1hdkM444wyn2S_CA0lwjqgW8nVnudzC2B9DlF-odFRbMk9YZ9OJTQLSlTyR66TM8o0EnEFouab4Dkn_ppqBASqb8NFmqoUgM/file?dl=1
Resolving ucd2b9fafdeb908c5b582f7b7515.dl.dropboxusercontent.com (ucd2b9fafdeb908c5b582f7b7515.dl.dropboxusercontent.com)... 162.125.13.15, 2620:100:6057:15::a27d:d0f

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Archive:  model.zip
  inflating: data-rb-size/data-00000-of-00001.arrow  
  inflating: data-rb-size/__MACOSX/._data-00000-of-00001.arrow  
  inflating: data-rb-size/dataset_info.json  
  inflating: data-rb-size/__MACOSX/._dataset_info.json  
  inflating: data-rb-size/state.json  
  inflating: data-rb-size/__MACOSX/._state.json  


In [15]:
from datasets import load_from_disk
dataset = load_from_disk('data-rb-size')
dataset = dataset.train_test_split(test_size=4/len(dataset))

dataset

DatasetDict({
    train: Dataset({
        features: ['svg', 'html'],
        num_rows: 95
    })
    test: Dataset({
        features: ['svg', 'html'],
        num_rows: 4
    })
})

In [16]:
data_prompt = """Your job is to take an SVG file of a web design and convert it into a pixel-perfect HTML and CSS markup and stylesheet.

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompt(examples):
    inputs       = examples["svg"]
    outputs      = examples["html"]
    texts = []
    for input_, output in zip(inputs, outputs):
        text = data_prompt.format(input_, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

training_data = dataset.map(formatting_prompt, batched=True)

Map:   0%|          | 0/95 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [17]:
training_data

DatasetDict({
    train: Dataset({
        features: ['svg', 'html', 'text'],
        num_rows: 95
    })
    test: Dataset({
        features: ['svg', 'html', 'text'],
        num_rows: 4
    })
})

In [18]:
trainer=SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=training_data['train'],
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=True,
    args=TrainingArguments(
        learning_rate=3e-4,
        lr_scheduler_type="linear",
        per_device_train_batch_size=16,
        gradient_accumulation_steps=8,
        num_train_epochs=40,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        warmup_steps=10,
        output_dir="output",
        seed=0,
    ),
)

trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 6 | Num Epochs = 40
O^O/ \_/ \    Batch size per device = 16 | Gradient Accumulation steps = 8
\        /    Total batch size = 128 | Total steps = 40
 "-____-"     Number of trainable parameters = 11,272,192


Step,Training Loss
1,0.229400
2,0.229400
3,0.222900
4,0.187600
5,0.195700
6,0.180400
7,0.152700
8,0.120000
9,0.106500
10,0.089600


TrainOutput(global_step=40, training_loss=0.0673998954705894, metrics={'train_runtime': 45.0734, 'train_samples_per_second': 5.325, 'train_steps_per_second': 0.887, 'total_flos': 7116163035955200.0, 'train_loss': 0.0673998954705894, 'epoch': 40.0})

In [21]:
text = dataset['test'][0]['svg']
model = FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    data_prompt.format(
        #instructions
        text,
        #answer
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 5020, use_cache = True)
answer=tokenizer.batch_decode(outputs)
answer = answer[0].split("### Response:")[-1]

print(text)
print(dataset['test'][0]['html'])
print("Answer of the question is:", answer)

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="393" height="852" viewBox="0 0 393 852"><g id="html1"><g data-tag="head" id="head1" data-z-index="auto" data-stacking-context="true" aria-owns="script1"><g data-tag="script" id="script1" data-z-index="auto" data-stacking-context="true"/></g><g data-tag="body" id="body1" data-z-index="auto" data-stacking-context="true" role="document" aria-owns="div1 style1"><g data-tag="div" id="div1" data-z-index="auto" data-stacking-context="true"><g data-stacking-layer="rootBackgroundAndBorders"><rect width="82.5234375" height="852" x="0" y="0" fill="rgb(0, 0, 0)"/></g></g><g data-tag="style" id="style1" data-z-index="auto" data-stacking-context="true"/></g></g></svg>
<body><div></div></body>

<style>


        body {
            margin: 0;
        }

        div {
            background-color: black;
            position: absolute;
            height: 100vh;
            width: 21%;
            top: 0;
        